<a href="https://colab.research.google.com/github/gpriya32/text-summarization/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [45]:
f=open("/content/sample_data/text.txt",'r')
text=f.read()
print(text)
f.close()

Those Who Are Resilient Stay In The Game Longer

“On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow.” — Friedrich Nietzsche
Challenges and setbacks are not meant to defeat you, but promote you. However, I realise after many years of defeats, it can crush your spirit and it is easier to give up than risk further setbacks and disappointments. Have you experienced this before? To be honest, I don’t have the answers. I can’t tell you what the right course of action is; only you will know. However, it’s important not to be discouraged by failure when pursuing a goal or a dream, since failure itself means different things to different people. To a person with a Fixed Mindset failure is a blow to their self-esteem, yet to a person with a Growth Mindset, it’s an opportunity to improve and find new ways to overcome their obstacles. Same failure, yet different

In [46]:
def text_cleaner(text):
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
        {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''}  # remove spaces at the beginning
    ]
    for rule in rules:
      for (k, v) in rule.items():
          regex = re.compile(k)
          text = regex.sub(v, text)
      text = text.rstrip()
    return text
    

In [47]:
ps=PorterStemmer()
def steming(words):
  stem=[]
  # play,played..
  for word in words:
    stem.append(ps.stem(word))
  # print(stem)
  return stem

In [48]:
# preprocessing 
text=text_cleaner(text)
# print(text)
sent_tokens=sent_tokenize(text)
word_tokens=word_tokenize(text)
word_tokens=[word.lower() for word in word_tokens]

# removing stopwords
stopwords=list(set(stopwords.words('english')))
word_token_ref=[word for word in word_tokens if word not in stopwords]

word_final=steming(word_token_ref)

In [49]:
freq={}
for word in word_final:
  if word in freq:
    freq[word]+=1
  else:
    freq[word]=1
print(len(word_tokens))
print(len(freq))

1214
331


In [50]:
def stemSentence(sentence):
    sentence=sentence.lower()
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(ps.stem(word))
        stem_sentence.append(" ")
    return stem_sentence

In [51]:
# computing the score of each sentence

def sentence_score():
      sentval={}
      sumval=0
      for sentence in sent_tokens:
        sentval[sentence]=0
        for word, count in freq.items():
          sent=stemSentence(sentence)
          if word in sent:
            sentval[sentence]+=count
      maxi=max(sentval.values())
      for key,val in sentval.items():
        sentval[key]=val/maxi
      print(sentval.values())
      return sentval


In [52]:
# from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
# lemmatizer = WordNetLemmatizer()
# print(lemmatizer.lemmatize("cats"))

In [53]:
# calculating cue -phrase

def cue_phrase_cal():
    cue_phrase=['firstly','secondly','thirdly','assumption','anyway','conclusion','nutshell','moreover']
    qphrase={}
    for sent in sent_tokens:
      qphrase[sent]=0
      tokens_word=word_tokenize(sent)
      for word in word_tokens:
        if word.lower() in cue_phrase:
          qphrase[sent]+=1

    maxi=max(qphrase.values())

    for key,val in qphrase.items():
      try:
        qphrase[key]=val/maxi
      except: 
        pass
    print(qphrase.values())
    return qphrase

In [54]:
# calculating upper case 

def upper_cal():
    upper={}
    for sent in sent_tokens:
      upper[sent]=0
      tokens_word=word_tokenize(sent)
      for word in word_tokens:
        if word.isupper():
          upper[sent]+=1

    maxi=max(upper.values())

    for key,val in upper.items():
      try:
        upper[key]=val/maxi
      except: 
        pass
    print(upper.values())
    return upper

In [55]:
# calculating upper case 

def digit_cal():
    digit={}
    for sent in sent_tokens:
      digit[sent]=0
      tokens_word=word_tokenize(sent)
      for word in word_tokens:
        if word.isdigit():
          digit[sent]+=1

    maxi=max(digit.values())

    for key,val in digit.items():
      try:
        digit[key]=val/maxi
      except: 
        pass
    print(digit.values())
    return digit

In [64]:
# calculating proper noun 
from nltk.tag import pos_tag

def pnoun_cal():
    pnoun={}
    for sent in sent_tokens:
      pnoun[sent]=0
      tagged_sent=pos_tag(sent.split())
      proper=[word for word,tag in tagged_sent if tag=='NNP']
      pnoun[sent]=len(proper)

    maxi=max(pnoun.values())

    for key,val in pnoun.items():
      try:
        pnoun[key]=val/maxi
      except: 
        pass
    print(pnoun.values())
    return pnoun

In [57]:
def sentence_len_cal():
    sent_len={}
    for sent in sent_tokens:
      sent_len[sent]=0
      word_tokens=word_tokenize(sent)
      length=len(word_tokens)
      if length in range(0,10):
        sent_len[sent]=1-0.05*(10-length)
      elif length in range(10,20):
        sent_len[sent]=1
      else:
        sent_len[sent]=1-0.05*(length-20)
    print(sent_len.values())
    return sent_len

In [58]:
def sentence_pos():
  sentence_position={}
  n=1
  N=len(sent_tokens)
  for sent in sent_tokens:
    a=1/n
    b=1/(N+1-n)
    sentence_position[sent]=max(a,b)
    n=n+1
  print(sentence_position.values())
  return sentence_position

In [59]:
# calculating upper case 

def heading_cal():
    head={}
    head_tokens=stemSentence(sent_tokens[0])

    for sent in sent_tokens:
      head[sent]=0
      word_tokens=stemSentence(sent)

      for word in word_tokens:
        if word not in stopwords and word in head_tokens:
          head[sent]+=1

    maxi=max(head.values())

    for key,val in head.items():
      try:
        head[key]=val/maxi
      except: 
        pass
    print(head.values())
    return head

In [60]:
import math

def create_frequency_matrix(sentences):
    frequency_matrix = {}

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopwords:
                pass
            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent] = freq_table

    return frequency_matrix


def create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix


def create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
          # occureance of word in sentences
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table


def create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
  # total_documents=total_sentences
  # inverse document frequency
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix


def create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix


def score_sent(tf_idf_matrix):

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += 0.5*score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue


def tfidf():
    sentences = sent_tokens
    total_documents = len(sentences)

    freq_matrix = create_frequency_matrix(sentences)
    #print(freq_matrix)

    tf_matrix = create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    count_doc_per_words = create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)

    idf_matrix = create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    
    tf_idf_matrix = create_tf_idf_matrix(tf_matrix, idf_matrix)

    tfidf_score = score_sent(tf_idf_matrix)

    print(tfidf_score.values())
    return tfidf_score



In [61]:
scores_fun=[sentence_score(),cue_phrase_cal(),upper_cal(),digit_cal(),sentence_pos(),sentence_len_cal(),pnoun_cal(),tfidf(),heading_cal()]
total_score={}
for fun in scores_fun:
  arr=fun
  for sent in sent_tokens:
    total_score[sent]=0
    if sent in arr:
      total_score[sent]+=arr[sent]

avg=np.mean(list(total_score.values()))


dict_values([0.6592920353982301, 0.4690265486725664, 0.035398230088495575, 0.47345132743362833, 0.3805309734513274, 0.6283185840707964, 0.5929203539823009, 0.4026548672566372, 0.05309734513274336, 0.18141592920353983, 0.24336283185840707, 0.4778761061946903, 0.19911504424778761, 0.2168141592920354, 0.7964601769911505, 0.5575221238938053, 0.7876106194690266, 0.22566371681415928, 0.3938053097345133, 0.29646017699115046, 1.0, 0.3893805309734513, 0.19911504424778761, 0.2345132743362832, 0.40707964601769914, 0.4424778761061947, 0.24778761061946902, 0.09734513274336283, 0.061946902654867256, 0.1504424778761062, 0.40707964601769914, 0.49557522123893805, 0.42035398230088494, 0.8053097345132744, 0.336283185840708, 0.13716814159292035, 0.3805309734513274, 0.23893805309734514, 0.5575221238938053, 0.33185840707964603, 0.4247787610619469, 0.4646017699115044, 0.18584070796460178, 0.21238938053097345, 0.5796460176991151, 0.49557522123893805, 0.336283185840708])
dict_values([0, 0, 0, 0, 0, 0, 0, 0, 0,

In [63]:
summary=""
for sentence in sent_tokens:
  if total_score[sentence]>1.5*avg:
    summary+=sentence+" "
print(summary)


Those Who Are Resilient Stay In The Game Longer “On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow.” — Friedrich Nietzsche Challenges and setbacks are not meant to defeat you, but promote you. It was the 19th Century’s minister Henry Ward Beecher who once said: “One’s best success comes after their greatest disappointments.” No one knows what the future holds, so your only guide is whether you can endure repeated defeats and disappointments and still pursue your dream. Consider the advice from the American academic and psychologist Angela Duckworth who writes in Grit: The Power of Passion and Perseverance: “Many of us, it seems, quit what we start far too early and far too often. Even more than the effort a gritty person puts in on a single day, what matters is that they wake up the next day, and the next, ready to get on that treadmill and keep goin